In [106]:
import numpy as np
import pandas as pd
import datetime as dt

### importing merge df

In [107]:
from _01_merged_df import MergedDataFrame 
obj = MergedDataFrame(underlying='BANKNIFTY')
merged_df = obj.get_merged_df()
merged_df

,open,high,low,close,ATMStrPr,currWeek_exp_dt,nextWeek_exp_dt,farWeek_exp_dt,currMonth_exp_dt,nextMonth_exp_dt,farMonth_exp_dt
datetime,,,,,,,,,,,
2020-01-01 09:15:00,32227.80,32295.80,32220.90,32292.00,32300.0,2020-01-02,2020-01-09,2020-01-16,2020-01-30,2020-02-27,2020-03-26
2020-01-01 09:16:00,32296.35,32303.30,32283.95,32288.25,32300.0,2020-01-02,2020-01-09,2020-01-16,2020-01-30,2020-02-27,2020-03-26
2020-01-01 09:17:00,32285.05,32290.85,32279.35,32282.95,32300.0,2020-01-02,2020-01-09,2020-01-16,2020-01-30,2020-02-27,2020-03-26
2020-01-01 09:18:00,32280.20,32296.25,32270.70,32292.45,32300.0,2020-01-02,2020-01-09,2020-01-16,2020-01-30,2020-02-27,2020-03-26
2020-01-01 09:19:00,32295.30,32312.05,32294.85,32298.65,32300.0,2020-01-02,2020-01-09,2020-01-16,2020-01-30,2020-02-27,2020-03-26
...,...,...,...,...,...,...,...,...,...,...,...
2022-12-19 15:25:00,43432.50,43433.35,43421.35,43429.75,43400.0,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-19 15:26:00,43428.05,43435.30,43425.95,43433.90,43400.0,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-19 15:27:00,43436.65,43451.45,43432.80,43444.60,43400.0,NaN,NaN,NaN,NaN,NaN,NaN


In [108]:
from datetime import datetime
underlying = 'BANKNIFTY'
date = '2021-01-01'
dateStr = date.split('-')[0]
time = '09:19'
exp_period = 'currWeek_exp_dt'
OTM_points = 0.2

dateTimeStr = date+' '+time
datetime_obj = datetime.strptime(dateTimeStr, '%Y-%m-%d %H:%M')
timeDt = datetime_obj.time()
dateDt = datetime_obj.date()

In [109]:
datetime_obj.time()

datetime.time(9, 19)

In [110]:
pd.to_datetime(time).time()

datetime.time(9, 19)

###  expiryDate formatting for symbol

In [111]:
expDate = merged_df.loc[datetime_obj,exp_period]
expDate = expDate.strftime('%y%m%d')


In [112]:
# merged_df.insert(4,'ATMStrPr',round(merged_df['close'],-2))

# def CE_strike_prices(entry_time_dt,date_dt):
#     return str(round(merged_df.loc[(merged_df.index.time == entry_time_dt) & (merged_df.index.date == date_dt), 'ATMStrPr'].values[0] + OTM_points))

# def PE_strike_prices(entry_time_dt,date_dt):
#     return str(round(merged_df.loc[(merged_df.index.time == entry_time_dt) & (merged_df.index.date == date_dt), 'ATMStrPr'].values[0] - OTM_points))


### strike prices setup for symbol

In [113]:
def CE_strike_price(date,time):
    return str(round(merged_df.loc[(merged_df.index.time == timeDt) & (merged_df.index.date == dateDt), 'ATMStrPr'].values[0] + OTM_points))

CE_strikePrice = CE_strike_price(dateDt,timeDt)

In [114]:
def PE_strike_price(date,time):
    return str(round(merged_df.loc[(merged_df.index.time == timeDt) & (merged_df.index.date == dateDt), 'ATMStrPr'].values[0] - OTM_points))

PE_strikePrice = PE_strike_price(dateDt,timeDt)

In [123]:
CE_strikePrice

'31300'

In [124]:
PE_strikePrice

'31300'

### symbol

In [116]:
CE_symbol = underlying+expDate+CE_strikePrice+'CE'

In [117]:
PE_symbol = underlying+expDate+PE_strikePrice+'PE'

### reading call price value

In [118]:
rel_CE_df = pd.read_csv(f"C:\\keshav\\Rahul\\Options_data\\NIFTY & BANKNIFTY (Jan 2020 to 19 Dec 2022)\\NIFTY & BANKNIFTY Options (Jan 2020 to 19 Dec 2022)\\{dateStr}\\{underlying} Options\\{CE_symbol}.csv",header=None, usecols=[0, 1, 2, 3, 4, 5], names=['date', 'time', 'open', 'high', 'low', 'close'])

rel_CE_df['date'] = pd.to_datetime(rel_CE_df['date'],format='%Y%m%d').dt.date
rel_CE_df['date'] = pd.to_datetime(rel_CE_df['date'])
rel_CE_df['date'] = rel_CE_df['date'].astype('str')
datetime = pd.to_datetime(rel_CE_df['date'] + ' ' + rel_CE_df['time'], format='%Y-%m-%d %H:%M')
rel_CE_df.set_index(datetime,inplace=True)

In [119]:
call_price = rel_CE_df.loc[datetime_obj,'close']
call_price

445.8

### reading put price value

In [120]:
rel_PE_df = pd.read_csv(f"C:\\keshav\\Rahul\\Options_data\\NIFTY & BANKNIFTY (Jan 2020 to 19 Dec 2022)\\NIFTY & BANKNIFTY Options (Jan 2020 to 19 Dec 2022)\\{dateStr}\\{underlying} Options\\{PE_symbol}.csv",header=None, usecols=[0, 1, 2, 3, 4, 5], names=['date', 'time', 'open', 'high', 'low', 'close'])

rel_PE_df['date'] = pd.to_datetime(rel_PE_df['date'],format='%Y%m%d').dt.date
rel_PE_df['date'] = pd.to_datetime(rel_PE_df['date'])
rel_PE_df['date'] = rel_PE_df['date'].astype('str')
datetime = pd.to_datetime(rel_PE_df['date'] + ' ' + rel_PE_df['time'], format='%Y-%m-%d %H:%M')
rel_PE_df.set_index(datetime,inplace=True)

In [121]:
rel_PE_df

,date,time,open,high,low,close
2021-01-01 09:15:00,2021-01-01,09:15,550.00,551.05,413.00,419.20
2021-01-01 09:16:00,2021-01-01,09:16,419.30,419.30,401.90,418.65
2021-01-01 09:17:00,2021-01-01,09:17,418.65,424.00,414.30,416.35
2021-01-01 09:18:00,2021-01-01,09:18,415.75,416.20,402.00,403.10
2021-01-01 09:19:00,2021-01-01,09:19,404.10,408.50,387.10,387.10
...,...,...,...,...,...,...
2021-01-07 15:25:00,2021-01-07,15:25,0.10,0.15,0.10,0.10
2021-01-07 15:26:00,2021-01-07,15:26,0.10,0.15,0.10,0.10
2021-01-07 15:27:00,2021-01-07,15:27,0.15,0.15,0.10,0.15
2021-01-07 15:28:00,2021-01-07,15:28,0.15,0.15,0.05,0.05


In [122]:
put_price = rel_PE_df.loc[datetime_obj,'close']
put_price

387.1